#Тестирование аугментаций Albumentations

###Зависимости

In [ ]:
#@markdown Импорт библиотек
import IPython.display as ipd
from albumentations import *
import cv2
import os
import torch
from shutil import copyfile
from tqdm import tqdm
import splitfolders
from pathlib import Path
ipd.clear_output()

###Аугментации

In [ ]:
#@markdown Туман
aug_1 = Compose([
    RandomFog(always_apply=False, p=1.0, fog_coef_lower=0.26, fog_coef_upper=0.46, alpha_coef=0.58),
    GaussNoise(always_apply=False, p=1.0, var_limit=(10.0, 101.06), per_channel=True, mean=1.42),],
    bbox_params=BboxParams(format='yolo'))

In [ ]:
#@markdown Работа с освещением
aug_2 = Compose([
    CLAHE(always_apply=False, p=1.0, clip_limit=(1, 9), tile_grid_size=(7, 9)),
    ColorJitter(always_apply=False, p=1.0, brightness=(0.8, 1.46), contrast=(0.8, 1.43), saturation=(0.12, 0.93), hue=(-0.2, 0.06)),],
    bbox_params=BboxParams(format='yolo'))

In [ ]:
#@markdown Дождь
aug_3 = Compose([
    RandomRain(always_apply=False, p=1.0, slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(0, 0, 0), blur_value=3, brightness_coefficient=0.77, rain_type=None),],
    bbox_params=BboxParams(format='yolo'))

In [ ]:
#@markdown Оптические искажения
aug_5 = Compose([
    OpticalDistortion(always_apply=False, p=1.0, distort_limit=(0.0, 0.3), shift_limit=(-0.09, 0.18), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),],
    bbox_params=BboxParams(format='yolo'))

###Работа с данными

In [ ]:
#@markdown Загрузка изображения
def imageLoad(path):
    image = cv2.imread(path)
    return image

In [ ]:
#@markdown
common_path = './project-11-at-2024-05-17-17-46-fasdqe18'
path_images = os.path.join(common_path, 'images')
path_labels = os.path.join(common_path, 'labels')

In [ ]:
#@markdown
for filename in tqdm(os.listdir(path_images)):
    image = imageLoad(os.path.join(path_images, filename))

    with open(os.path.join(path_labels, f'{Path(filename).stem}.txt')) as label_file:
        annotations = label_file.readlines()
        bboxes = []
        for el in annotations:
            if el == '\n':
                continue
            bboxes.append(list(map(float, el.split()[1:])))
            bboxes[-1].append(el.split()[0])

    augmentations = [aug_1, aug_2, aug_3, aug_4, aug_5]
    for i, el in enumerate(augmentations):
        new = el(image=image, bboxes=bboxes)
        new_image = new['image']
        new_bboxes = new['bboxes']
        labels = []
        for bbox in new_bboxes:
            class_annotation = bbox[-1]
            bbox = list(map(str, bbox[:-1]))
            bbox.insert(0, class_annotation)
            labels.append(" ".join(bbox) + '\n')
        new_name = f'{Path(filename).stem}-{i + 1}'
        cv2.imwrite(os.path.join(path_images, f'{new_name}.jpg'), new_image)
        with open(os.path.join(path_labels, f'{new_name}.txt'), 'w') as new_label:
            new_label.writelines(labels)
